In [1]:
cd /anaconda3/envs/nlp/lib/python3.7/site-packages

/anaconda3/envs/nlp/lib/python3.7/site-packages


In [2]:
import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *
import pandas as pd

In [3]:
import MeCab
from os import path
import neologdn
import re
import emoji

pos_list = [10, 11, 31, 32, 34]
pos_list.extend(list(range(36,50)))
pos_list.extend([59, 60, 62, 67])

def create_mecab_list(text):
    mecab_list = []
    mecab = MeCab.Tagger("-Ochasen")
    mecab.parse("")
    # encoding = text.encode('utf-8')
    node = mecab.parseToNode(text)
    while node:
        if len(node.surface) > 1:
            if node.posid in pos_list:
                morpheme = node.surface
                mecab_list.append(morpheme)
        node = node.next
    return mecab_list

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    cleaned_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return cleaned_text

def clean_text(text):
    text = give_emoji_free_text(text)
    text = neologdn.normalize(text)
    text = create_mecab_list(text)    
    return text

In [9]:
no_of_reviews = 10000
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

In [10]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True
reviews = pd.DataFrame(columns = ["review", "Author Name", "Review Date", "Review Ratings", 
                                  "Review Body", "Developer Reply"])
temp = {"review": 0, "Author Name": "", "Review Date": "", "Review Ratings": 0, 
        "Review Body": "", "Developer Reply": ""}

def replace_value_with_definition(key_to_find, definition):
    for key in temp.keys():
        if key == key_to_find:
            temp[key] = definition

In [11]:
cd ~/Desktop/chrome

/Users/justinwang/Desktop/chrome


In [12]:
driver = webdriver.Chrome(r"./chromedriver")
wait = WebDriverWait(driver, 10)

# Append your app store urls here
urls = ["https://play.google.com/store/apps/details?id=com.skysoft.kkbox.android&hl=ja"]

for url in urls:

    driver.get(url)

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")

    expatistan_table = soup_expatistan.find("h1", class_="AHFaub")

    print("App name: ", expatistan_table.string)

    expatistan_table = soup_expatistan.findAll("span", class_="htlgb")[4]

    print("Installs Range: ", expatistan_table.string)

    expatistan_table = soup_expatistan.find("meta", itemprop="ratingValue")

    print("Rating Value: ", expatistan_table['content'])

    expatistan_table = soup_expatistan.find("meta", itemprop="reviewCount")

    print("Reviews Count: ", expatistan_table['content'])

    soup_histogram = soup_expatistan.find("div", class_="VEF2C")

    rating_bars = soup_histogram.find_all('div', class_="mMF0fd")

    for rating_bar in rating_bars:
        print("Rating: ", rating_bar.find("span").text)
        print("Rating count: ", rating_bar.find("span", class_="L2o20d").get('title'))

    # open all reviews
    url = url + '&showAllReviews=true'
    driver.get(url)
    time.sleep(5) # wait dom ready
    for i in range(1,100):
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # scroll to load other reviews
            time.sleep(2)
            if check_exists_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/content/span'):
                driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/content/span').click()
                time.sleep(2)
        except:
            pass

    page = driver.page_source

    soup_expatistan = BeautifulSoup(page, "html.parser")
    expand_pages = soup_expatistan.findAll("div", class_="d15Mdf")
    counter = 1
    items = []
    
    for expand_page in expand_pages:
        try:
            # print("\n===========\n")
            
            # print("review：" + str(counter))
            review = str(counter)
            
            # print("Author Name: ", str(expand_page.find("span", class_="X43Kjb").text))
            Author_Name = str(expand_page.find("span", class_="X43Kjb").text)
            
            # print("Review Date: ", expand_page.find("span", class_="p2TkOb").text)
            Review_Date = str(expand_page.find("span", class_="p2TkOb").text)
            
            '''
            //didn't find reviewer link
            print("Reviewer Link: ", expand_page.find("a", class_="reviews-permalink")['href'])
            '''
            
            reviewer_ratings = expand_page.find("div", class_="pf5lIe").find_next()['aria-label'];
            reviewer_ratings = reviewer_ratings.split('/')[0]
            reviewer_ratings = ''.join(x for x in reviewer_ratings if x.isdigit())
            # print("Reviewer Ratings: ", reviewer_ratings)
            Reviewer_Ratings = int(reviewer_ratings)
            
            '''
            //didn't find review title
            print("Review Title: ", str(expand_page.find("span", class_="review-title").string))
            '''
            
            # print("Review Body: ", str(expand_page.find("div", class_="UD7Dzf").text))
            Review_Body = str(expand_page.find("div", class_="UD7Dzf").text)
            Review_Body_cleaned = clean_text(Review_Body)
            Review_Body_string = ''.join(Review_Body_cleaned)
            
            developer_reply = expand_page.find_parent().find("div", class_="LVQB0b")
            if hasattr(developer_reply, "text"):
                # print("Developer Reply: " + "\n", str(developer_reply.text))
                Developer_Reply = str(developer_reply.text)
            else:
                # print("Developer Reply: ", "")
                Developer_Reply = ""
            
            counter += 1           
            item = {
                    "review": counter - 1,
                    "Author Name": Author_Name,
                    "Reviewer Ratings": Reviewer_Ratings,
                    "Review Date": Review_Date,
                    "Review Body": Review_Body_string,
                    "Developer Reply": Developer_Reply
                    }
            items.append(item)
                
        except:
            pass
driver.quit()

App name:  KKBOX - 1ヶ月無料 音楽聞き放題アプリ
Installs Range:  10,000,000+
Rating Value:  4.149554252624512
Reviews Count:  351638
Rating:  5
Rating count:  None
Rating:  4
Rating count:  None
Rating:  3
Rating count:  None
Rating:  2
Rating count:  None
Rating:  1
Rating count:  None


In [13]:
df = pd.DataFrame(items, columns = ["review", "Author Name", "Review Date", "Reviewer Ratings", 
                                    "Review Body", "Developer Reply"])
df

,review,Author Name,Review Date,Reviewer Ratings,Review Body,Developer Reply
0,1,宍戸日香里,2019年4月27日,1,ふざけんかなり解約いまだに毎月980しはらわ解約方法送っアプリダウンロード手続きしよ退会使っ...,KKBOX International Limited2019年4月30日ご利用いただき誠に...
1,2,michy9807,2019年4月15日,4,台湾使用ランキング特定アーティスト偏っくわ多い値段安い使い,
2,3,小杉菜々香,2019年4月6日,5,リンゴマークアプリ乗り換え不具合原因分から良けれアプリ使っ月額料金高い聴き大抵あっ操作わかり...,
3,4,匿名希望,2019年5月3日,1,トライアル期間登録瞬間終わる無料なかっ詐欺アプリ一生アプリ使う無い,KKBOX International Limited2019年5月7日大変お手数ではござい...
4,5,Googleユーザー,2019年3月25日,1,解約月額支払い発生詐欺使っお金とらお金返し,
5,6,takao ramen,2019年3月9日,5,アプリ使い最近これ使っアプリスリープタイマーイコライザー内蔵ワイヤレスイヤホン聞く音質調整で...,
6,7,Googleユーザー,2019年3月25日,5,かれこれ使っ最新もちろん8090聴い不具合特になく普通動い月額料金安く値段使っ満足あり家族薦め,
7,8,ミハラコタロウ,2019年4月19日,5,以前動作異様重たく困っ最近アップデート解消やっぱり使い,
8,9,*萌,2019年5月5日,5,使い思い(*´｀*),
9,10,Googleユーザー,2019年2月14日,4,最近KKbox聴こ変わっもしかして設定ミス月額料金払わ思っ支払い履歴設定確認問題原因よく分か...,KKBOX International Limited2019年2月14日大変お手数ではござ...


In [14]:
import numpy as np
df['Review Body'].replace('', np.nan, inplace=True)
df.dropna(subset=['Review Body'], inplace=True)
df

,review,Author Name,Review Date,Reviewer Ratings,Review Body,Developer Reply
0,1,宍戸日香里,2019年4月27日,1,ふざけんかなり解約いまだに毎月980しはらわ解約方法送っアプリダウンロード手続きしよ退会使っ...,KKBOX International Limited2019年4月30日ご利用いただき誠に...
1,2,michy9807,2019年4月15日,4,台湾使用ランキング特定アーティスト偏っくわ多い値段安い使い,
2,3,小杉菜々香,2019年4月6日,5,リンゴマークアプリ乗り換え不具合原因分から良けれアプリ使っ月額料金高い聴き大抵あっ操作わかり...,
3,4,匿名希望,2019年5月3日,1,トライアル期間登録瞬間終わる無料なかっ詐欺アプリ一生アプリ使う無い,KKBOX International Limited2019年5月7日大変お手数ではござい...
4,5,Googleユーザー,2019年3月25日,1,解約月額支払い発生詐欺使っお金とらお金返し,
5,6,takao ramen,2019年3月9日,5,アプリ使い最近これ使っアプリスリープタイマーイコライザー内蔵ワイヤレスイヤホン聞く音質調整で...,
6,7,Googleユーザー,2019年3月25日,5,かれこれ使っ最新もちろん8090聴い不具合特になく普通動い月額料金安く値段使っ満足あり家族薦め,
7,8,ミハラコタロウ,2019年4月19日,5,以前動作異様重たく困っ最近アップデート解消やっぱり使い,
8,9,*萌,2019年5月5日,5,使い思い(*´｀*),
9,10,Googleユーザー,2019年2月14日,4,最近KKbox聴こ変わっもしかして設定ミス月額料金払わ思っ支払い履歴設定確認問題原因よく分か...,KKBOX International Limited2019年2月14日大変お手数ではござ...


In [15]:
cd ~/Desktop

/Users/justinwang/Desktop


In [16]:
df.to_csv("KKBOX_Reviews.csv", columns = ["review", "Author Name", "Review Date", "Reviewer Ratings", 
          "Review Body", "Developer Reply"])

In [18]:
df_KKBOX = pd.read_csv("GooglePlayReviews.csv")
df_UtaPass = pd.read_csv("KKBOX_Reviews.csv")

frames = [df_KKBOX, df_UtaPass]
reviews_kkstream = pd.concat(frames)

In [21]:
reviews_kkstream.to_csv("reviews_kkstream.csv", columns = ["review", "Author Name", "Review Date", "Reviewer Ratings", 
                        "Review Body", "Developer Reply"])